In [23]:
import pandas as pd
import openpyxl
import xlrd

# Load the data
stock_data = pd.read_excel('mohtly-return-cap-NEW.xlsx', engine = 'openpyxl')
market_data = pd.read_excel('market-return.xlsx', engine = 'openpyxl')
risk_free_data = pd.read_excel('risk-free.xlsx', engine = 'openpyxl')

# Check the first few rows to understand the data
print(stock_data.head())
print(market_data.head())
print(risk_free_data.head())


FileNotFoundError: [Errno 2] No such file or directory: 'mohtly-return-cap-NEW.xlsx'